# The aim of this script is to process the xvg data including total and non-polar SASA over the simulation, calculate the average, save the data into a dataframe


- for the SASA calculation, do I use e.g. the last 50ns frames for average or the whole frames? I guess it depends on what you want and what you wish to analyse. If the starting period where you still haven’t reached the equilibrium is important for the mechnism you are looking at. I guess then it’s not. Better to get rid of the first 50ns

In [1]:
# To create a reusable function that reads XVG data from a file and saves it into a dataframe
# Of course it is not perfect. The column names are not specified since rmsd, rmsf, gyrate differs, and it is a bit tricky to specify it since gyrate has four columns instead of two like the other two


import pandas as pd

def read_xvg(file_path):
    data = []

    with open(file_path, 'r') as f:
        for line in f:
            if not line.startswith("#") and not line.startswith('@'):
                new_list = [elem for elem in line.split()]
                data.append(new_list)
    # Convert the list of lists into a Pandas DataFrame
    df = pd.DataFrame(data)
    return df

### Total SASA (last 50ns)

In [47]:
import os
import pandas as pd
import fnmatch # looking for a specific string in a file name
import re
import statistics

xvgpath = 'AllXVGfiles_totalSASA_noH251023/' # change it everytime if you have a new␣ ↪directory
# reference: https://stackoverflow.com/questions/55245721/ ↪read-in-multiple-csv-into-separate-dataframes-in-pandas
# https://stackoverflow.com/questions/36294712/ ↪python-filename-contains-string-metachar
# https://www.delftstack.com/howto/python/python-fnmatch/
# save rmsd, rmsf, gyrate files into different dictionaries


directory = xvgpath

# sasa files
sasa = [x for x in os.listdir(directory) if fnmatch.fnmatch(x, '*.xvg')] # fnmatch look for rmsd filename specifically


## example for fnmatch: the files that start with hello and end with .js `if␣ ↪fnmatch.fnmatch(file, "hello*.js"):`
d_sasa = {} # dictionary that will hold them 


for file_name in sasa: # loop over files
    
    # read data into a dataframe and add it to dict with file_name as it key
    df_sasa = read_xvg(os.path.join(directory,file_name)) # to note, directory and file_name need to be joint or else the file can't be␣ ↪found
    
    # drop the first 50ns 
    df_sasa = df_sasa.drop(range(0,501))
    
    # convert all columns to numeric
    df_sasa = df_sasa.apply(pd.to_numeric)
    
    # calculate the average over the frames
    df_sasa = df_sasa.mean(axis =0)
    
    # save it into a dictionary
    d_sasa[file_name] = df_sasa[1]

    
# read 54 conditions into a variable

run54 = pd.read_excel("54ConditionName120923.xlsx",index_col=0)

run54 = run54['Condition']


# loop over 54 conditions

sasa_avg = []

#loop over 54 conditions
for i in run54: 
   
    replicas = []
    
    # averaging replicas
    for key in d_sasa:
        
        if re.search(i, key): # be very careful with the variable when you use it with the wildcard together. You should use it like␣ ↪it is in the left. https://www.tutorialspoint.com/ ↪how-to-use-variables-in-python-regular-expression
            
            # if find a match, append the dataframe to the list
            replicas.append(d_sasa[key])
  

    # calculate the mean of the list
    average_replicas = statistics.mean(replicas)

    # append the mean value for each condition to a list
    sasa_avg.append(average_replicas)

    
# concat list elements to a dataframe, combine by columns
df_avg = pd.DataFrame(sasa_avg)

# transpose to add columns
df_avg = df_avg.T


# Assing column names
df_avg.columns = run54
df_avg = df_avg.T
df_avg.columns = ['Total SASA']

# save the data into an excel file
# df_avg.to_excel("TotalSASA281023.xlsx")

### non-polar SASA (last 50ns)

In [50]:
import os
import pandas as pd
import fnmatch # looking for a specific string in a file name
import re
import statistics

xvgpath = 'AllXVGfiles_NonPolar251023/' # change it everytime if you have a new␣ ↪directory
# reference: https://stackoverflow.com/questions/55245721/ ↪read-in-multiple-csv-into-separate-dataframes-in-pandas
# https://stackoverflow.com/questions/36294712/ ↪python-filename-contains-string-metachar
# https://www.delftstack.com/howto/python/python-fnmatch/
# save rmsd, rmsf, gyrate files into different dictionaries


directory = xvgpath

# sasa files
sasa = [x for x in os.listdir(directory) if fnmatch.fnmatch(x, '*.xvg')] # fnmatch look for rmsd filename specifically


## example for fnmatch: the files that start with hello and end with .js `if␣ ↪fnmatch.fnmatch(file, "hello*.js"):`
d_sasa = {} # dictionary that will hold them 


for file_name in sasa: # loop over files
    
    # read data into a dataframe and add it to dict with file_name as it key
    df_sasa = read_xvg(os.path.join(directory,file_name)) # to note, directory and file_name need to be joint or else the file can't be␣ ↪found
    
    # drop the first 50ns 
    df_sasa = df_sasa.drop(range(0,501))
    
    # convert all columns to numeric
    df_sasa = df_sasa.apply(pd.to_numeric)
    
    # calculate the average over the frames
    df_sasa = df_sasa.mean(axis =0)
    
    # save it into a dictionary
    d_sasa[file_name] = df_sasa[1]

    
# read 54 conditions into a variable

run54 = pd.read_excel("54ConditionName120923.xlsx",index_col=0)

run54 = run54['Condition']


# loop over 54 conditions

sasa_avg = []

#loop over 54 conditions
for i in run54: 
   
    replicas = []
    
    # averaging replicas
    for key in d_sasa:
        
        if re.search(i, key): # be very careful with the variable when you use it with the wildcard together. You should use it like␣ ↪it is in the left. https://www.tutorialspoint.com/ ↪how-to-use-variables-in-python-regular-expression
            
            # if find a match, append the dataframe to the list
            replicas.append(d_sasa[key])
  

    # calculate the mean of the list
    average_replicas = statistics.mean(replicas)

    # append the mean value for each condition to a list
    sasa_avg.append(average_replicas)

    
# concat list elements to a dataframe, combine by columns
df_avg = pd.DataFrame(sasa_avg)

# transpose to add columns
df_avg = df_avg.T


# Assing column names
df_avg.columns = run54
df_avg = df_avg.T
df_avg.columns = ['Non_polar SASA']

# save the data into an excel file
df_avg.to_excel("NonPolarSASA281023.xlsx")

### join the two dataframes together

In [56]:
# read data from an excel
non_polar_sasa = pd.read_excel("NonPolarSASA281023.xlsx",index_col=0)
total_sasa = pd.read_excel("TotalSASA281023.xlsx",index_col=0)

# concatnate the two datasets

df = pd.concat([non_polar_sasa, total_sasa], axis = 1)

# save the dataframe to an excel file
df.to_excel("SASA281023.xlsx")